In [ ]:
import pandas as pd
import time
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
import gresearch_crypto
import gc
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

TRAIN_CSV = '/kaggle/input/g-research-crypto-forecasting/train.csv'
ASSET_DETAILS_CSV = '/kaggle/input/g-research-crypto-forecasting/asset_details.csv'

def read_csv_strict(file_name='/kaggle/input/g-research-crypto-forecasting/train.csv'):
    df = pd.read_csv(file_name)
    df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')
    df = df[df['datetime'] > '2019-12-1 00:00:00' ].set_index('timestamp')
    df = df[df['datetime'] < '2021-06-13 00:00:00' ]
    
    return df

In [ ]:
df_train = read_csv_strict()
asset=[]
for i in range(14):
    asset.append(df_train[df_train['Asset_ID']==i])
del df_train

FE

In [ ]:
def log_return(series, periods=1):
    return np.log1p(series).diff(periods=periods)
def get_features(data):
    
    data['Upper_Shadow'] = data['High'] - np.maximum(data['Close'], data['Open'])
    data['Lower_Shadow'] = np.minimum(data['Close'], data['Open']) - data['Low']
    
    
    data['volume2count'] = data['Volume'] / (data['Count'] + 1)
    
    data['Low_15min']=(pd.concat([data['Low'].shift(i) for i in range(16)],axis=1)).min(axis=1)
    data['High_15min']=(pd.concat([data['High'].shift(i) for i in range(16)],axis=1)).max(axis=1)

    data['Upper_Shadow_15min'] = data['High_15min'] - np.maximum(data['Close'], data['Open'])
    data['Lower_Shadow_15min'] = np.minimum(data['Close'], data['Open']) - data['Low_15min']
    
    for i in [1,2,15,16]:
        data['Target-'+str(i)]=data['Target'].shift(i)
    
    for i in [1,15,45,60]:
        data['lr'+str(i)]=log_return(data['Close'], periods=i)
        data['lrv'+str(i)]=log_return(data['VWAP'], periods=i)
    
    data['Close_position']=(data['Close']-data['Low'])/(data['High']-data['Low']+1)
    data['Close_position_15min']=(data['Close']-data['Low_15min'])/(data['High_15min']-data['Low_15min']+1)
    
    data['Open_position']=(data['Open']-data['Low'])/(data['High']-data['Low']+1)
    data['Open_position_15min']=(data['Open']-data['Low_15min'])/(data['High_15min']-data['Low_15min']+1)   
    
    data['h/h15']=(data['High']/data['High_15min']).transform(np.log1p)
    data['l/l15']=(data['Low']/data['Low_15min']).transform(np.log1p)
    for c in ['High','Low','VWAP','High_15min','Low_15min']:
        data['Close/'+c]=(data['Close']/data[c]).transform(np.log1p)
    for c in ['Open','High','Low','Close','VWAP','High_15min','Low_15min']:
        data[c]=data[c].transform(np.log1p)
    
    return 0

In [ ]:
for a in asset :
    get_features(a)
    a.replace(-np.inf, np.nan,inplace=True)
    a.replace(np.inf, np.nan,inplace=True)
    a.dropna(inplace=True)

In [ ]:
features=asset[1].columns.difference(['Asset_ID','Target','datetime'])

In [ ]:
x=[]
for i in range (14):
    x.append( asset[i].loc[:, features].values)
    x[i] = StandardScaler().fit_transform(x[i])

In [ ]:
principal_df=[]
for k in range(14):
    pca_df = PCA(0.95)
    principalComponents_df = pca_df.fit_transform(x[k])
    principal_df .append( pd.DataFrame(data = principalComponents_df
                 , columns = ['feature '+str(i) for i in range(1,principalComponents_df.shape[1]+1)]))

In [ ]:
for k in range(14):
    principal_df[k]['timestamp']=asset[k].reset_index()['timestamp']
    principal_df[k]=principal_df[k].set_index('timestamp')
del x

In [ ]:
y=[]
for i in range(14):
    y.append(asset[i].Target)

In [ ]:
models=[]
for i in range(14):
    models.append(LinearRegression())

In [ ]:

for i in range(14):
    models[i].fit(principal_df[i][:200000],y[i][:200000])
        

In [ ]:
preds=[]
for i in range(14):
    pred=(models[i].predict(principal_df[i][200000:210000]))
    preds.append(pred)

In [ ]:
for i in range(14):
    pred=pd.Series(preds[i].reshape(10000,))
    true=pd.Series(y[i][200000:210000].reset_index().Target)
    print('for asset'+str(i))
    print(pd.DataFrame({'pred':pred,'true':true}).corr().iloc[1,0])

In [ ]:
import matplotlib.pyplot as plt
i=500
x=240000
plt.plot(principal_df[1][x:x+i].index,y[1][x:x+i])
plt.plot(principal_df[1][x:x+i].index, models[1].predict(principal_df[1][x:x+i]))

In [ ]:
i=500
x=240000
plt.plot(principal_df[0][x:x+i].index,y[0][x:x+i])
plt.plot(principal_df[0][x:x+i].index, models[0].predict(principal_df[0][x:x+i]))